In [46]:
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from pycaret.time_series import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pycaret.utils.time_series import clean_time_index
import holidays
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

# อ่านข้อมูลจากไฟล์ CSV
df = pd.read_csv(
    "daily_csv/export-jsps014-1d.csv", parse_dates=["timestamp"], index_col="timestamp"
)
df = df.drop(columns=["Unnamed: 0"], errors="ignore")
print("Index เป็น unique หรือไม่:", df.index.is_unique)

df = df[:]
# ลบ index ที่ซ้ำกันและตั้งค่าความถี่เป็นรายวัน
df = df[~df.index.duplicated(keep="last")]
df = df.interpolate(method="spline", order=2)
df = df.asfreq("D")

# กำหนดเกณฑ์สำหรับค่า pm_2_5
df = df[(df["pm_2_5"] >= 0) & (df["pm_2_5"] <= 100)]
df = df[(df["pm_2_5_sp"] >= 0) & (df["pm_2_5_sp"] <= 130)]

# ใช้เฉพาะคอลัมน์ที่ต้องการ
df = df[["pm_2_5", "pm_2_5_sp", "temperature", "humidity"]]  # เพิ่ม temperature และ humidity

# # สร้าง Lag Features สำหรับ pm_2_5_sp
# for lag in range(15, 30):
#     df[f"pm_2_5_sp_lag{lag}"] = df["pm_2_5_sp"].shift(lag)

# # สร้าง Lag Features สำหรับ pm_2_5
# for lag in range(15, 30):
#     df[f"pm_2_5_lag{lag}"] = df["pm_2_5"].shift(lag)

# # สร้าง Lag Features สำหรับ temperature และ humidity
# for lag in range(15, 30):
#     df[f"temperature_lag{lag}"] = df["temperature"].shift(lag)
#     df[f"humidity_lag{lag}"] = df["humidity"].shift(lag)

# เพิ่มฟีเจอร์ วัน, เดือน, ปี
df['day'] = df.index.day
df['month'] = df.index.month
df['year'] = df.index.year

# เพิ่มฟีเจอร์ "ฤดูกาล" ของประเทศไทย
def get_thai_season(month):
    if month in [3, 4, 5]:
        return 'summer'
    elif month in [6, 7, 8, 9, 10]:
        return 'rainy'
    else:
        return 'winter'

df['season'] = df['month'].apply(get_thai_season)

# One-hot encoding สำหรับฤดูกาล
df = pd.get_dummies(df, columns=['season'], prefix='season')

# เพิ่มฟีเจอร์เกี่ยวกับลมมรสุม
def get_monsoon_features(month):
    if month in [6, 7, 8, 9, 10]:  # ฤดูฝน (ลมมรสุมตะวันตกเฉียงใต้)
        return 1, 0, 0
    elif month in [11, 12, 1, 2]:  # ฤดูหนาว (ลมมรสุมตะวันออกเฉียงเหนือ)
        return 0, 1, 0
    else:  # ช่วงเปลี่ยนผ่าน (เมษายน-พฤษภาคม)
        return 0, 0, 1

df[['monsoon_sw', 'monsoon_ne', 'transition_period']] = df['month'].apply(
    lambda x: pd.Series(get_monsoon_features(x))
)

# เพิ่มฟีเจอร์ "อิทธิพลจากไฟป่าอินโดนีเซีย"
def get_indonesia_fire(month):
    if month in [8, 9, 10]:  # อิทธิพลจากไฟป่าอินโดนีเซีย
        return 1
    else:
        return 0

df['indonesia_fire'] = df['month'].apply(get_indonesia_fire)

# เพิ่มฟีเจอร์ "ช่วงที่ลมหยุดนิ่ง" (Calm Wind Period)
def get_calm_wind_period(month):
    if month in [4, 5, 10, 11]:  # ช่วงเปลี่ยนผ่านมรสุม
        return 1
    else:
        return 0

df['calm_wind_period'] = df['month'].apply(get_calm_wind_period)

# เพิ่มฟีเจอร์ "ช่วงปิดเทอมและเปิดเทอม"
def get_school_term(month):
    if month in [5, 6, 7, 8, 9]:  # เปิดเทอม 1 (กลางพฤษภาคม - ต้นตุลาคม)
        return 1
    elif month in [11, 12, 1, 2]:  # เปิดเทอม 2 (ต้นพฤศจิกายน - ปลายกุมภาพันธ์)
        return 1
    else:  # ปิดเทอม
        return 0

def get_university_term(month):
    if month in [8, 9, 10, 11, 12]:  # เปิดเทอม 1 (สิงหาคม - ธันวาคม)
        return 1
    elif month in [1, 2, 3, 4, 5]:  # เปิดเทอม 2 (มกราคม - พฤษภาคม)
        return 1
    else:  # ปิดเทอม
        return 0

df['school_term'] = df['month'].apply(get_school_term)
df['university_term'] = df['month'].apply(get_university_term)

# เพิ่มฟีเจอร์ "ความแรงลม" (Wind Level)
def get_wind_level(month):
    if month in [6, 7, 8, 9]:  # ลมแรงมาก (Level 4)
        return 4
    elif month in [5, 10]:  # ลมแรง (Level 3)
        return 3
    elif month in [1, 2, 12]:  # ลมปานกลาง (Level 2)
        return 2
    elif month in [3, 4, 11]:  # ลมอ่อน (Level 1)
        return 1
    else:  # ลมสงบ (Level 0)
        return 0

df['wind_level'] = df['month'].apply(get_wind_level)

# เพิ่มฟีเจอร์ "ระดับการผลิต" ของโรงงาน 3 ประเภท
def get_agriculture_production(month):
    if month in [1, 2, 3]:  # มกราคม - มีนาคม (เก็บเกี่ยวผลผลิต)
        return 4
    elif month in [9, 10, 11, 12]:  # กันยายน - ธันวาคม (ปลูกพืช)
        return 3
    else:  # เมษายน - สิงหาคม (ฤดูฝน, การผลิตลดลง)
        return 1

def get_automotive_production(month):
    if month in [1, 2, 3]:  # มกราคม - มีนาคม (เริ่มปีใหม่, การผลิตเพิ่มขึ้น)
        return 4
    elif month in [10, 11, 12]:  # ตุลาคม - ธันวาคม (สิ้นปี, การผลิตเพิ่มขึ้น)
        return 3
    else:  # เมษายน - กันยายน (ฤดูฝน, การผลิตลดลง)
        return 2

def get_consumer_goods_production(month):
    if month in [1, 2, 3]:  # มกราคม - มีนาคม (เทศกาลปีใหม่, การผลิตเพิ่มขึ้น)
        return 4
    elif month in [10, 11, 12]:  # ตุลาคม - ธันวาคม (เทศกาลสิ้นปี, การผลิตเพิ่มขึ้น)
        return 3
    else:  # เมษายน - กันยายน (ฤดูฝน, การผลิตลดลง)
        return 2

df['agriculture_production'] = df['month'].apply(get_agriculture_production)
df['automotive_production'] = df['month'].apply(get_automotive_production)
df['consumer_goods_production'] = df['month'].apply(get_consumer_goods_production)

# เพิ่มฟีเจอร์ "การขนส่งสินค้า" (Freight Transport)
def get_freight_transport(month):
    if month in [1, 4, 5, 6, 7, 8]:  # เดือนที่มีการขนส่งสูง
        return 4
    elif month in [2, 3, 9, 10]:  # เดือนที่มีการขนส่งปานกลาง
        return 3
    else:  # เดือนที่มีการขนส่งต่ำ
        return 2

df['freight_transport'] = df['month'].apply(get_freight_transport)

# เพิ่มฟีเจอร์ "การผลิตพลังงาน" (Energy Production)
def get_energy_production(month):
    if month in [1, 2, 3, 12]:  # เดือนที่มีการใช้พลังงานสูง
        return 4
    elif month in [4, 5, 6, 7]:  # เดือนที่มีการใช้พลังงานปานกลาง
        return 3
    else:  # เดือนที่มีการใช้พลังงานต่ำ
        return 2

df['energy_production'] = df['month'].apply(get_energy_production)

# เพิ่มฟีเจอร์ "วันหยุด" ของประเทศไทย
thai_holidays = holidays.TH(years=df.index.year.unique())
df['is_holiday'] = df.index.to_series().apply(lambda x: 1 if x in thai_holidays else 0)

# แปลงประเภทข้อมูลของคอลัมน์ให้เป็นตัวเลข
df['is_holiday'] = df['is_holiday'].astype(int)
df['season_summer'] = df['season_summer'].astype(int)
df['season_rainy'] = df['season_rainy'].astype(int)
df['season_winter'] = df['season_winter'].astype(int)
df['monsoon_sw'] = df['monsoon_sw'].astype(int)
df['monsoon_ne'] = df['monsoon_ne'].astype(int)
df['transition_period'] = df['transition_period'].astype(int)
df['indonesia_fire'] = df['indonesia_fire'].astype(int)
df['calm_wind_period'] = df['calm_wind_period'].astype(int)
df['school_term'] = df['school_term'].astype(int)
df['university_term'] = df['university_term'].astype(int)
df['wind_level'] = df['wind_level'].astype(int)
df['agriculture_production'] = df['agriculture_production'].astype(int)
df['automotive_production'] = df['automotive_production'].astype(int)
df['consumer_goods_production'] = df['consumer_goods_production'].astype(int)
df['freight_transport'] = df['freight_transport'].astype(int)
df['energy_production'] = df['energy_production'].astype(int)

# ใช้เฉพาะคอลัมน์ที่ต้องการ
df = df[["pm_2_5", "pm_2_5_sp", "temperature", "humidity"]]  # เพิ่ม temperature และ humidity

# สร้าง Lag Features สำหรับ pm_2_5_sp
for lag in range(15, 45,2):
    df[f"pm_2_5_sp_lag{lag}"] = df["pm_2_5_sp"].shift(lag)

# สร้าง Lag Features สำหรับ pm_2_5
for lag in range(15, 45,2):
    df[f"pm_2_5_lag{lag}"] = df["pm_2_5"].shift(lag)

# สร้าง Lag Features สำหรับ temperature และ humidity
for lag in range(15, 45,2):
    df[f"temperature_lag{lag}"] = df["temperature"].shift(lag)
    df[f"humidity_lag{lag}"] = df["humidity"].shift(lag)

# Shift pm_2_5 ไป 15 วันก่อน
df["pm_2_5_shift15"] = df["pm_2_5"].shift(15)
df["pm_2_5_sp_shift15"] = df["pm_2_5_sp"].shift(15)
df["temperature_shift15"] = df["temperature"].shift(15)
df["humidity_shift15"] = df["humidity"].shift(15)

windows = range(1, 30,2)  # ตั้งแต่ 1 ถึง 17 วัน

# คำนวณค่าเฉลี่ยย้อนหลังโดยใช้ลูป
for window in windows:
    df[f"pm_2_5_ma{window}"] = df["pm_2_5"].shift(15).rolling(window=window).mean()
    df[f"pm_2_5_sp_ma{window}"] = df["pm_2_5_sp"].shift(15).rolling(window=window).mean()
    df[f"temperature_ma{window}"] = df["temperature"].shift(15).rolling(window=window).mean()
    df[f"humidity_ma{window}"] = df["humidity"].shift(15).rolling(window=window).mean()

# ลบคอลัมน์ shift ที่ไม่ต้องการออก
df = df.drop(columns=["pm_2_5_shift15"])
df = df.drop(columns=["pm_2_5_sp_shift15"])
df = df.drop(columns=["temperature_shift15"])
df = df.drop(columns=["humidity_shift15"])

# ลบคอลัมน์ pm_2_5_sp ที่ไม่ต้องการใช้
df = df.drop(columns="pm_2_5_sp")
df = df[-1000:]
print(df)

# Ensure the DataFrame has a DateTime index with a frequency
df = df.asfreq("D")  # Set frequency to daily

# Fill any missing values after setting the frequency
df = df.interpolate(method="linear", limit_direction="forward")

# Split the data into train and test sets
train_size = len(df) - 14
train_df = df.iloc[:train_size]
test_df = df.iloc[train_size:]

# Ensure the train and test sets have a frequency
train_df = train_df.asfreq("D")
test_df = test_df.asfreq("D")

# ลบคอลัมน์ที่ไม่ต้องการใช้
train_df = train_df.drop(columns=["temperature", "humidity",'freight_transport','indonesia_fire','season_summer','season_rainy','season_winter','agriculture_production','automotive_production','consumer_goods_production','monsoon_sw','monsoon_ne','transition_period'])
test_df = test_df.drop(columns=["pm_2_5", "temperature", "humidity",'freight_transport','indonesia_fire','season_summer','season_rainy','season_winter','agriculture_production','automotive_production','consumer_goods_production','monsoon_sw','monsoon_ne','transition_period'])


# Set up PyCaret
exp = TSForecastingExperiment()
exp.setup(
    data=train_df,  # Use train_df with a defined frequency
    target="pm_2_5",
    session_id=123,
    fh=14,  # Forecast horizon
    use_gpu=True,
)

# Create and finalize the ARIMA model
model = exp.create_model("arima", order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
model = exp.finalize_model(model)

# แปลงดัชนีของ test_df เป็น PeriodIndex ก่อนทำนาย
test_df_period = test_df.copy()
test_df_period.index = test_df_period.index.to_period("D")

# Make predictions
forecast = exp.predict_model(model, fh=14, X=test_df_period)

# Ensure predictions are non-negative
forecast["y_pred"] = np.maximum(forecast["y_pred"], 0)

# แปลงดัชนีของ forecast กลับเป็น datetime64[ns] เพื่อการพล็อตกราฟ
forecast.index = forecast.index.to_timestamp()

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(df.index[-50:], df["pm_2_5"][-50:], label="Actual", marker="o")
plt.plot(
    forecast.index,
    forecast["y_pred"],
    label="Forecast",
    marker="s",
    linestyle="dashed",
)
plt.xlabel("Date")
plt.ylabel("pm_2_5")
plt.title

Index เป็น unique หรือไม่: True
                        pm_2_5  temperature   humidity  pm_2_5_sp_lag15  \
timestamp                                                                 
2022-05-17 07:00:00  15.788506    27.948805  72.038097        21.110274   
2022-05-18 07:00:00   9.481538    30.198423  67.144211        18.678202   
2022-05-19 07:00:00  24.730858    30.807471  65.339858        12.097507   
2022-05-20 07:00:00  14.737404    31.920448  57.033495        20.946168   
2022-05-21 07:00:00  11.706139    31.173422  59.517832        19.034483   
...                        ...          ...        ...              ...   
2025-02-07 07:00:00  19.059025    29.077198  68.224688        31.186038   
2025-02-08 07:00:00  19.064854    29.716301  65.974686        22.123305   
2025-02-09 07:00:00  13.824501    29.145664  66.850779        17.109808   
2025-02-10 07:00:00  18.744632    29.690582  67.651647        26.194777   
2025-02-11 07:00:00  25.979693    29.804851  66.932706        10.158

KeyError: "['freight_transport', 'indonesia_fire', 'season_summer', 'season_rainy', 'season_winter', 'agriculture_production', 'automotive_production', 'consumer_goods_production', 'monsoon_sw', 'monsoon_ne', 'transition_period'] not found in axis"

In [ ]:
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    mean_absolute_percentage_error,
    r2_score
)
import numpy as np

# ดึงค่าจริงจาก test_df
actual = df[-14:]['pm_2_5']

# ดึงค่าทำนายจาก forecast
predicted = forecast['y_pred']

# คำนวณ MAE
mae = mean_absolute_error(actual, predicted)

# คำนวณ MSE
mse = mean_squared_error(actual, predicted)

# คำนวณ RMSE
rmse = np.sqrt(mse)

# คำนวณ MAPE โดยใช้ฟังก์ชันจาก sklearn
mape = mean_absolute_percentage_error(actual, predicted) * 100  # แปลงเป็นเปอร์เซ็นต์

# คำนวณ R²
r2 = r2_score(actual, predicted)

# คำนวณความแม่นยำ (Accuracy)
mean_actual = np.mean(actual)
accuracy = 100 - mape 

# แสดงผลลัพธ์
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAPE: {mape:.2f}%")
print(f"R²: {r2:.4f}")
print(f"ความแม่นยำ (Accuracy): {accuracy:.2f}%")

MAE: 6.6562
MSE: 84.5313
RMSE: 9.1941
MAPE: 33.29%
R²: -0.6395
ความแม่นยำ (Accuracy): 66.71%
